# 9. Consolidate observations

## Setup

In [1]:
library(tidyverse)

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1       ✔ purrr   0.3.2  
✔ tibble  2.1.1       ✔ dplyr   0.8.0.1
✔ tidyr   0.8.3       ✔ stringr 1.4.0  
✔ readr   1.3.1       ✔ forcats 0.4.0  
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
sample_file <- '../preprocessed_data/sample_2019-09-10.feather'
company_groups_file <- '../preprocessed_data/company_groups_2019-09-10.feather'
m_as_file <- '../preprocessed_data/m_as_2019-09-10.feather'

## Read data

In [3]:
sample <- feather::read_feather(sample_file)
testit::assert(nrow(sample[duplicated(select(sample, YEAR, COMMODITY, OPERATOR_ID)), ]) == 0)
glimpse(sample)

Warning message:
“Coercing int64 to double”

Observations: 3,200
Variables: 20
$ OPERATOR_ID           <chr> "300", "300", "300", "300", "300", "300", "300"…
$ YEAR                  <dbl> 2017, 2017, 2011, 2014, 2012, 2010, 2018, 2018,…
$ NAME                  <chr> "PLAINS PIPELINE, L.P.", "PLAINS PIPELINE, L.P.…
$ COMMODITY             <chr> "hvl", "non-hvl", "non-hvl", "non-hvl", "non-hv…
$ MILES                 <dbl> 72.900, 0.000, 326.800, NA, 265.170, 62.280, 29…
$ AGE_UNKNOWN_MILES     <dbl> 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,…
$ MILES_PRE_1940        <dbl> 0.00, 3.72, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,…
$ MILES_1940            <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,…
$ MILES_1950            <dbl> 0.000, 0.000, 0.800, 0.000, 0.800, 244.980, 0.0…
$ MILES_1960            <dbl> 0.00, 0.00, 188.43, 0.00, 188.43, 4.76, 0.00, 0…
$ MILES_1970            <dbl> 0.00, 0.00, 1.17, 0.00, 1.17, 0.00, 0.00, 0.00,…
$ MILES_1980            <dbl> 0.00, 0.00, 4.43, 0.00, 4.43, 0.00, 0.00, 0.00,…
$ MILES_1990      

In [4]:
company_groups <- feather::read_feather(company_groups_file)
glimpse(company_groups)

Observations: 135
Variables: 2
$ members <chr> "30829", "32209", "3445", "32109", "30629", "15485", "31684",…
$ name    <chr> "Teppco (Group)", "Teppco (Group)", "Teppco (Group)", "ONEOK …


In [5]:
m_as_orig <- feather::read_feather(m_as_file)
glimpse(m_as_orig)

Observations: 38
Variables: 4
$ members    <chr> "Enterprise Products (Group)", "Teppco (Group)", "Sunoco (…
$ name       <chr> "Enterprise Products (Group)", "Enterprise Products (Group…
$ start_year <chr> "2010", "2010", "2011", "2011", "2011", "2013", "2013", "2…
$ end_year   <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, "2013", "2013"…


## 9.1 Calculate change

In [6]:
sample <- sample %>%
    group_by(OPERATOR_ID, COMMODITY) %>%
    arrange(YEAR) %>%
    mutate(CHANGE = MILES - lag(MILES, 1))
glimpse(subset(sample, YEAR > 2010))

Observations: 1,805
Variables: 21
Groups: OPERATOR_ID, COMMODITY [277]
$ OPERATOR_ID           <chr> "300", "395", "473", "473", "994", "1248", "124…
$ YEAR                  <dbl> 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011,…
$ NAME                  <chr> "PLAINS PIPELINE, L.P.", "AMOCO OIL CO", "ANADA…
$ COMMODITY             <chr> "non-hvl", "non-hvl", "crude", "hvl", "hvl", "c…
$ MILES                 <dbl> 326.800, 527.000, 12.000, NA, 121.400, 126.000,…
$ AGE_UNKNOWN_MILES     <dbl> 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000…
$ MILES_PRE_1940        <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,…
$ MILES_1940            <dbl> 0.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 801.9,…
$ MILES_1950            <dbl> 0.800, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000…
$ MILES_1960            <dbl> 188.430, 1.000, 43.000, 0.000, 121.000, 187.000…
$ MILES_1970            <dbl> 1.170, 2.000, 16.000, 0.000, 1.000, 189.000, 0.…
$ MILES_1980            <dbl> 4.430, 17.000, 0.000, 2.000, 1

## 9.2 Consolidate observations

### 9.2.1 Company groups

In [7]:
company_groups <- rename(company_groups, new_name = name)
sample <- left_join(sample, company_groups, by=c('OPERATOR_ID' = 'members'))
testit::assert(nrow(sample[duplicated(select(sample, YEAR, COMMODITY, OPERATOR_ID)), ]) == 0)
glimpse(subset(sample, !is.na(sample$new_name)))

Observations: 2,096
Variables: 22
Groups: OPERATOR_ID, COMMODITY [222]
$ OPERATOR_ID           <chr> "395", "994", "1845", "2162", "2170", "2190", "…
$ YEAR                  <dbl> 2004, 2004, 2004, 2004, 2004, 2004, 2004, 2004,…
$ NAME                  <chr> "BP PIPELINES NORTH AMERICA INC.", "WILLIAMS MI…
$ COMMODITY             <chr> "non-hvl", "crude", "non-hvl", "hvl", "non-hvl"…
$ MILES                 <dbl> 728.00, 0.00, 3047.00, 130.00, 365.00, 164.00, …
$ AGE_UNKNOWN_MILES     <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,…
$ MILES_PRE_1940        <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 13, 0, 0, 0, 0, 0, 0…
$ MILES_1940            <dbl> 15, 0, 729, 0, 0, 0, 0, 0, 0, 48, 0, 58, 0, 0, …
$ MILES_1950            <dbl> 108.0, 0.0, 567.0, 0.0, 70.4, 0.0, 65.0, 0.0, 4…
$ MILES_1960            <dbl> 1, 0, 581, 0, 188, 0, 0, 0, 139, 222, 1289, 135…
$ MILES_1970            <dbl> 5.0, 0.0, 0.0, 249.0, 23.1, 89.0, 0.0, 0.0, 320…
$ MILES_1980            <dbl> 26.0, 0.0, 0.0, 0.0, 2.7, 2.0,

In [8]:
sample_no_group <- subset(sample, is.na(sample$new_name))
sample_group <- subset(sample, !is.na(sample$new_name))

sample_group <- sample_group %>%
    group_by(new_name, YEAR, COMMODITY) %>%
    summarize(OPERATOR_ID = first(new_name), 
              NAME              = first(new_name),
              MILES             = sum(MILES, na.rm=TRUE), 
              AGE_UNKNOWN_MILES = sum(AGE_UNKNOWN_MILES, na.rm=TRUE), 
              MILES_PRE_1940    = sum(MILES_PRE_1940, na.rm=TRUE), 
              MILES_1940      = sum(MILES_1940, na.rm=TRUE),
              MILES_1950      = sum(MILES_1950, na.rm=TRUE),
              MILES_1960      = sum(MILES_1960, na.rm=TRUE),
              MILES_1970      = sum(MILES_1970, na.rm=TRUE),
              MILES_1980      = sum(MILES_1980, na.rm=TRUE),
              MILES_1990      = sum(MILES_1990, na.rm=TRUE),
              MILES_2000      = sum(MILES_2000, na.rm=TRUE),
              MILES_2010      = sum(MILES_2010, na.rm=TRUE),
              PARENT            = first(PARENT), 
              INCIDENTS         = sum(INCIDENTS, na.rm=TRUE), 
              SIGNIFICANT_INCIDENTS = sum(SIGNIFICANT_INCIDENTS, na.rm=TRUE), 
              GROUP = 'group') %>%
    # We calculate the avg age and change once all groupings (including M&As below), are taken care of
    ungroup()

glimpse(sample_group)

Observations: 1,131
Variables: 20
$ new_name              <chr> "Amoco (Group)", "Amoco (Group)", "Amoco (Group…
$ YEAR                  <dbl> 2004, 2004, 2005, 2005, 2006, 2006, 2007, 2007,…
$ COMMODITY             <chr> "hvl", "non-hvl", "hvl", "non-hvl", "hvl", "non…
$ OPERATOR_ID           <chr> "Amoco (Group)", "Amoco (Group)", "Amoco (Group…
$ NAME                  <chr> "Amoco (Group)", "Amoco (Group)", "Amoco (Group…
$ MILES                 <dbl> 665.94, 728.00, 665.94, 728.00, 665.94, 728.00,…
$ AGE_UNKNOWN_MILES     <dbl> 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,…
$ MILES_PRE_1940        <dbl> 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.…
$ MILES_1940            <dbl> 0.00, 15.00, 0.00, 15.00, 0.00, 15.00, 0.00, 15…
$ MILES_1950            <dbl> 0.50, 108.00, 0.50, 108.00, 0.50, 108.00, 0.25,…
$ MILES_1960            <dbl> 0.00, 1.00, 0.00, 1.00, 0.00, 1.00, 0.00, 1.00,…
$ MILES_1970            <dbl> 1349.44, 5.00, 1349.44, 5.00, 1349.44, 5.00, 14…
$ MILES_1980      

In [9]:
sample_no_group$GROUP = "not group"

sample <- bind_rows(sample_group, sample_no_group)
sample <- sample[ , (names(sample) != "new_name")]
testit::assert(nrow(sample[duplicated(select(sample, YEAR, COMMODITY, OPERATOR_ID)), ]) == 0)
glimpse(sample_n(sample, 10))

Observations: 10
Variables: 22
$ YEAR                  <dbl> 2012, 2009, 2018, 2018, 2006, 2015, 2016, 2011,…
$ COMMODITY             <chr> "crude", "crude", "non-hvl", "hvl", "non-hvl", …
$ OPERATOR_ID           <chr> "26049", "22855", "Torrance (Group)", "30626", …
$ NAME                  <chr> "COUNTRYMARK REFINING AND LOGISTICS, LLC", "KOC…
$ MILES                 <dbl> NA, 1097.000, 163.182, 233.770, 1130.679, 29.16…
$ AGE_UNKNOWN_MILES     <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
$ MILES_PRE_1940        <dbl> 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7
$ MILES_1940            <dbl> 56.60, 12.00, 5.45, 63.30, 0.00, 0.00, 105.83, …
$ MILES_1950            <dbl> 0.000, 209.000, 13.080, 83.100, 97.160, 0.000, …
$ MILES_1960            <dbl> 0.000, 0.000, 15.308, 31.800, 197.010, 0.000, 4…
$ MILES_1970            <dbl> 38.300, 264.000, 103.659, 37.930, 11.970, 0.000…
$ MILES_1980            <dbl> 0.000, 954.000, 5.872, 1.230, 18.510, 0.000, 46…
$ MILES_1990            <dbl> 34.200, 77.

In [10]:
testit::assert(nrow(subset(sample, YEAR == 2008)) > 0)
table(sample$GROUP)


    group not group 
     1131      1104 

### 9.2.2 M&As

In [11]:
m_as <- rename(m_as_orig, new_name = name)

# We need to fill in the NA values for start and end_year with values that always match, because when logical conditions encounter NAs
# they will always be wrong. The filtering in those cases happens because the new_name column is NA.
m_as[is.na(m_as$start_year), ]$start_year <- -9999
m_as[is.na(m_as$end_year), ]$end_year <- 9999

years = data.frame(year = unique(sample$YEAR))
m_as <- merge(m_as, years)
m_as <- m_as %>%
    filter(year >= start_year & year < end_year) %>%
    select(members, new_name, year) %>%
    distinct()
m_as$m_a = TRUE

glimpse(m_as)

Observations: 284
Variables: 4
$ members  <chr> "Enbridge (Group)", "15774", "BP (Group)", "30781", "Chevron…
$ new_name <chr> "Enbridge (Group)", "Enbridge (Group)", "BP (Group)", "BP (G…
$ year     <dbl> 2004, 2004, 2004, 2004, 2004, 2004, 2004, 2004, 2004, 2005, …
$ m_a      <lgl> TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, …


In [12]:
sample <- left_join(sample, m_as, by=c('OPERATOR_ID' = 'members', 'YEAR' = 'year'))
sample$m_a <- replace_na(sample$m_a, FALSE)

glimpse(sample)

Observations: 2,246
Variables: 24
$ YEAR                  <dbl> 2004, 2004, 2005, 2005, 2006, 2006, 2007, 2007,…
$ COMMODITY             <chr> "hvl", "non-hvl", "hvl", "non-hvl", "hvl", "non…
$ OPERATOR_ID           <chr> "Amoco (Group)", "Amoco (Group)", "Amoco (Group…
$ NAME                  <chr> "Amoco (Group)", "Amoco (Group)", "Amoco (Group…
$ MILES                 <dbl> 665.94, 728.00, 665.94, 728.00, 665.94, 728.00,…
$ AGE_UNKNOWN_MILES     <dbl> 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,…
$ MILES_PRE_1940        <dbl> 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.…
$ MILES_1940            <dbl> 0.00, 15.00, 0.00, 15.00, 0.00, 15.00, 0.00, 15…
$ MILES_1950            <dbl> 0.50, 108.00, 0.50, 108.00, 0.50, 108.00, 0.25,…
$ MILES_1960            <dbl> 0.00, 1.00, 0.00, 1.00, 0.00, 1.00, 0.00, 1.00,…
$ MILES_1970            <dbl> 1349.44, 5.00, 1349.44, 5.00, 1349.44, 5.00, 14…
$ MILES_1980            <dbl> 0.00, 26.00, 0.00, 26.00, 0.00, 26.00, 0.00, 26…
$ MILES_1990      

In [13]:
sample_m_as <- subset(sample, m_a == TRUE)
sample_no_m_as <- subset(sample, m_a == FALSE)

glimpse(sample_m_as)

Observations: 400
Variables: 24
$ YEAR                  <dbl> 2014, 2015, 2016, 2017, 2018, 2004, 2004, 2004,…
$ COMMODITY             <chr> "hvl", "hvl", "hvl", "hvl", "hvl", "crude", "hv…
$ OPERATOR_ID           <chr> "Boardwalk (Group)", "Boardwalk (Group)", "Boar…
$ NAME                  <chr> "Boardwalk (Group)", "Boardwalk (Group)", "Boar…
$ MILES                 <dbl> 179.30, 260.70, 262.00, 262.40, 268.10, 36.00, …
$ AGE_UNKNOWN_MILES     <dbl> 0.0, 0.0, 0.0, 3.3, 3.3, 0.0, 0.0, 0.0, 0.0, 0.…
$ MILES_PRE_1940        <dbl> 0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 152.0…
$ MILES_1940            <dbl> 0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 43.00…
$ MILES_1950            <dbl> 0.00, 0.00, 0.00, 0.00, 0.00, 4.00, 0.00, 498.0…
$ MILES_1960            <dbl> 319.90, 56.50, 185.70, 167.30, 165.80, 7.00, 28…
$ MILES_1970            <dbl> 0.00, 12.70, 12.90, 12.90, 12.90, 5.00, 631.00,…
$ MILES_1980            <dbl> 0.00, 0.00, 4.00, 4.00, 4.00, 2.00, 36.00, 39.0…
$ MILES_1990        

In [14]:
sample_m_as <- sample_m_as %>%
    group_by(new_name, YEAR, COMMODITY) %>%
    summarize(OPERATOR_ID = first(new_name), 
              NAME              = first(new_name),
              MILES             = sum(MILES, na.rm=TRUE), 
              AGE_UNKNOWN_MILES = sum(AGE_UNKNOWN_MILES, na.rm=TRUE), 
              MILES_PRE_1940    = sum(MILES_PRE_1940, na.rm=TRUE), 
              MILES_1940        = sum(MILES_1940, na.rm=TRUE),
              MILES_1950        = sum(MILES_1950, na.rm=TRUE),
              MILES_1960        = sum(MILES_1960, na.rm=TRUE),
              MILES_1970        = sum(MILES_1970, na.rm=TRUE),
              MILES_1980        = sum(MILES_1980, na.rm=TRUE),
              MILES_1990        = sum(MILES_1990, na.rm=TRUE),
              MILES_2000        = sum(MILES_2000, na.rm=TRUE),
              MILES_2010        = sum(MILES_2010, na.rm=TRUE),
              PARENT            = first(PARENT), 
              INCIDENTS         = sum(INCIDENTS, na.rm=TRUE), 
              SIGNIFICANT_INCIDENTS = sum(SIGNIFICANT_INCIDENTS, na.rm=TRUE), 
              GROUP = 'm&a') %>%
    ungroup()

glimpse(sample_m_as)

Observations: 277
Variables: 20
$ new_name              <chr> "Boardwalk (Group)", "Boardwalk (Group)", "Boar…
$ YEAR                  <dbl> 2014, 2015, 2016, 2017, 2018, 2004, 2004, 2004,…
$ COMMODITY             <chr> "hvl", "hvl", "hvl", "hvl", "hvl", "crude", "hv…
$ OPERATOR_ID           <chr> "Boardwalk (Group)", "Boardwalk (Group)", "Boar…
$ NAME                  <chr> "Boardwalk (Group)", "Boardwalk (Group)", "Boar…
$ MILES                 <dbl> 239.50, 330.40, 330.40, 330.66, 337.60, 36.00, …
$ AGE_UNKNOWN_MILES     <dbl> 0.0, 3.7, 3.7, 7.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.…
$ MILES_PRE_1940        <dbl> 0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 152.0…
$ MILES_1940            <dbl> 0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 43.00…
$ MILES_1950            <dbl> 163.90, 163.70, 163.50, 163.10, 163.30, 4.00, 0…
$ MILES_1960            <dbl> 319.90, 56.50, 185.70, 167.30, 165.80, 7.00, 28…
$ MILES_1970            <dbl> 0.00, 12.70, 12.90, 12.90, 12.90, 5.00, 631.00,…
$ MILES_1980        

In [15]:
sample <- bind_rows(sample_m_as, sample_no_m_as)
sample <- sample[ , !(names(sample) %in% c("new_name", "m_a"))]
testit::assert(nrow(sample[duplicated(select(sample, YEAR, COMMODITY, OPERATOR_ID)), ]) == 0)
testit::assert(nrow(subset(sample, YEAR == 2008)) > 0)
glimpse(sample)

Observations: 2,123
Variables: 22
$ YEAR                  <dbl> 2014, 2015, 2016, 2017, 2018, 2004, 2004, 2004,…
$ COMMODITY             <chr> "hvl", "hvl", "hvl", "hvl", "hvl", "crude", "hv…
$ OPERATOR_ID           <chr> "Boardwalk (Group)", "Boardwalk (Group)", "Boar…
$ NAME                  <chr> "Boardwalk (Group)", "Boardwalk (Group)", "Boar…
$ MILES                 <dbl> 239.50, 330.40, 330.40, 330.66, 337.60, 36.00, …
$ AGE_UNKNOWN_MILES     <dbl> 0.0, 3.7, 3.7, 7.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.…
$ MILES_PRE_1940        <dbl> 0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 152.0…
$ MILES_1940            <dbl> 0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 43.00…
$ MILES_1950            <dbl> 163.90, 163.70, 163.50, 163.10, 163.30, 4.00, 0…
$ MILES_1960            <dbl> 319.90, 56.50, 185.70, 167.30, 165.80, 7.00, 28…
$ MILES_1970            <dbl> 0.00, 12.70, 12.90, 12.90, 12.90, 5.00, 631.00,…
$ MILES_1980            <dbl> 0.00, 0.00, 4.00, 4.00, 4.00, 2.00, 36.00, 41.0…
$ MILES_1990      

## 9.3 Mark M&A years

In [16]:
glimpse(m_as_orig)

Observations: 38
Variables: 4
$ members    <chr> "Enterprise Products (Group)", "Teppco (Group)", "Sunoco (…
$ name       <chr> "Enterprise Products (Group)", "Enterprise Products (Group…
$ start_year <chr> "2010", "2010", "2011", "2011", "2011", "2013", "2013", "2…
$ end_year   <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, "2013", "2013"…


In [17]:
m_as_orig$start_year <- as.numeric(m_as_orig$start_year)
m_as_orig <- select(m_as_orig, -c(name))

m_as_start <- subset(m_as_orig, !is.na(start_year))
m_as_start$m_a_start <- TRUE

sample <- left_join(sample, m_as_start, by=c("OPERATOR_ID" = "members", "YEAR" = "start_year"))
testit::assert(nrow(subset(sample, YEAR == 2008)) > 0)
testit::assert(nrow(sample[duplicated(select(sample, YEAR, COMMODITY, OPERATOR_ID)), ]) == 0)

glimpse(sample)

Observations: 2,123
Variables: 24
$ YEAR                  <dbl> 2014, 2015, 2016, 2017, 2018, 2004, 2004, 2004,…
$ COMMODITY             <chr> "hvl", "hvl", "hvl", "hvl", "hvl", "crude", "hv…
$ OPERATOR_ID           <chr> "Boardwalk (Group)", "Boardwalk (Group)", "Boar…
$ NAME                  <chr> "Boardwalk (Group)", "Boardwalk (Group)", "Boar…
$ MILES                 <dbl> 239.50, 330.40, 330.40, 330.66, 337.60, 36.00, …
$ AGE_UNKNOWN_MILES     <dbl> 0.0, 3.7, 3.7, 7.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.…
$ MILES_PRE_1940        <dbl> 0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 152.0…
$ MILES_1940            <dbl> 0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 43.00…
$ MILES_1950            <dbl> 163.90, 163.70, 163.50, 163.10, 163.30, 4.00, 0…
$ MILES_1960            <dbl> 319.90, 56.50, 185.70, 167.30, 165.80, 7.00, 28…
$ MILES_1970            <dbl> 0.00, 12.70, 12.90, 12.90, 12.90, 5.00, 631.00,…
$ MILES_1980            <dbl> 0.00, 0.00, 4.00, 4.00, 4.00, 2.00, 36.00, 41.0…
$ MILES_1990      

In [18]:
m_as_orig$end_year <- as.numeric(m_as_orig$end_year)

m_as_end <- subset(m_as_orig, !is.na(end_year))
# We add 1 to the year, so the year we match on is the first year after the group has ceased to exist.
m_as_end$end_year <- m_as_end$end_year + 1
m_as_end$m_a_end <- TRUE
m_as_end <- unique(m_as_end)

sample <- left_join(sample, m_as_end, by=c("OPERATOR_ID" = "members", "YEAR" = "end_year"))
testit::assert(nrow(subset(sample, YEAR == 2008)) > 0)
testit::assert(nrow(sample[duplicated(select(sample, YEAR, COMMODITY, OPERATOR_ID)), ]) == 0)

glimpse(sample)

Observations: 2,123
Variables: 26
$ YEAR                  <dbl> 2014, 2015, 2016, 2017, 2018, 2004, 2004, 2004,…
$ COMMODITY             <chr> "hvl", "hvl", "hvl", "hvl", "hvl", "crude", "hv…
$ OPERATOR_ID           <chr> "Boardwalk (Group)", "Boardwalk (Group)", "Boar…
$ NAME                  <chr> "Boardwalk (Group)", "Boardwalk (Group)", "Boar…
$ MILES                 <dbl> 239.50, 330.40, 330.40, 330.66, 337.60, 36.00, …
$ AGE_UNKNOWN_MILES     <dbl> 0.0, 3.7, 3.7, 7.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.…
$ MILES_PRE_1940        <dbl> 0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 152.0…
$ MILES_1940            <dbl> 0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 43.00…
$ MILES_1950            <dbl> 163.90, 163.70, 163.50, 163.10, 163.30, 4.00, 0…
$ MILES_1960            <dbl> 319.90, 56.50, 185.70, 167.30, 165.80, 7.00, 28…
$ MILES_1970            <dbl> 0.00, 12.70, 12.90, 12.90, 12.90, 5.00, 631.00,…
$ MILES_1980            <dbl> 0.00, 0.00, 4.00, 4.00, 4.00, 2.00, 36.00, 41.0…
$ MILES_1990      

In [19]:
sample$M_A <- (sample$m_a_start | sample$m_a_end)
sample <- select(sample, -c(end_year, m_a_start, start_year, m_a_end))
sample$M_A <- replace_na(sample$M_A, FALSE)

glimpse(sample)

Observations: 2,123
Variables: 23
$ YEAR                  <dbl> 2014, 2015, 2016, 2017, 2018, 2004, 2004, 2004,…
$ COMMODITY             <chr> "hvl", "hvl", "hvl", "hvl", "hvl", "crude", "hv…
$ OPERATOR_ID           <chr> "Boardwalk (Group)", "Boardwalk (Group)", "Boar…
$ NAME                  <chr> "Boardwalk (Group)", "Boardwalk (Group)", "Boar…
$ MILES                 <dbl> 239.50, 330.40, 330.40, 330.66, 337.60, 36.00, …
$ AGE_UNKNOWN_MILES     <dbl> 0.0, 3.7, 3.7, 7.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.…
$ MILES_PRE_1940        <dbl> 0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 152.0…
$ MILES_1940            <dbl> 0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 43.00…
$ MILES_1950            <dbl> 163.90, 163.70, 163.50, 163.10, 163.30, 4.00, 0…
$ MILES_1960            <dbl> 319.90, 56.50, 185.70, 167.30, 165.80, 7.00, 28…
$ MILES_1970            <dbl> 0.00, 12.70, 12.90, 12.90, 12.90, 5.00, 631.00,…
$ MILES_1980            <dbl> 0.00, 0.00, 4.00, 4.00, 4.00, 2.00, 36.00, 41.0…
$ MILES_1990      

## 9.4 Calculate avg age, change for new groups

In [20]:
testit::assert(nrow(subset(sample, YEAR == 2008)) > 0)
sample <- sample %>%
    group_by(OPERATOR_ID, COMMODITY) %>%
    arrange(YEAR) %>%
    mutate(CHANGE = MILES - lag(MILES, 1))
glimpse(subset(sample, YEAR > 2010))

Observations: 1,176
Variables: 23
Groups: OPERATOR_ID, COMMODITY [173]
$ YEAR                  <dbl> 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011,…
$ COMMODITY             <chr> "hvl", "non-hvl", "crude", "hvl", "non-hvl", "h…
$ OPERATOR_ID           <chr> "Chevron (Group)", "Chevron (Group)", "Enbridge…
$ NAME                  <chr> "Chevron (Group)", "Chevron (Group)", "Enbridge…
$ MILES                 <dbl> 601.100, 943.700, 2197.200, 154.800, 682.500, 3…
$ AGE_UNKNOWN_MILES     <dbl> 6.10, 23.00, 0.00, 0.00, 0.00, 52.20, 0.00, 0.0…
$ MILES_PRE_1940        <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,…
$ MILES_1940            <dbl> 0.00, 85.00, 444.00, 0.00, 0.00, 245.00, 0.00, …
$ MILES_1950            <dbl> 833.300, 537.000, 293.000, 188.000, 135.000, 31…
$ MILES_1960            <dbl> 1053.700, 232.000, 615.000, 0.000, 290.000, 164…
$ MILES_1970            <dbl> 1066.500, 21.000, 7.000, 0.000, 86.000, 411.800…
$ MILES_1980            <dbl> 334.000, 19.000, 66.000, 0.000

In [21]:
sample$AVG_AGE <- ((sample$MILES_PRE_1940 * 90 +
                    sample$MILES_1940 * 75 +
                    sample$MILES_1950 * 65 +
                    sample$MILES_1960 * 55 +
                    sample$MILES_1970 * 45 +
                    sample$MILES_1980 * 35 +
                    sample$MILES_1990 * 25 +
                    sample$MILES_2000 * 15 +
                    sample$MILES_2010 * 05) /
                   (sample$MILES_PRE_1940 +
                    sample$MILES_1940 +
                    sample$MILES_1950 +
                    sample$MILES_1960 +
                    sample$MILES_1970 +
                    sample$MILES_1980 +
                    sample$MILES_1990 +
                    sample$MILES_2000 +
                    sample$MILES_2010))

glimpse(sample)

Observations: 2,123
Variables: 23
Groups: OPERATOR_ID, COMMODITY [193]
$ YEAR                  <dbl> 2004, 2004, 2004, 2004, 2004, 2004, 2004, 2004,…
$ COMMODITY             <chr> "crude", "hvl", "non-hvl", "hvl", "non-hvl", "h…
$ OPERATOR_ID           <chr> "BP (Group)", "BP (Group)", "BP (Group)", "Chev…
$ NAME                  <chr> "BP (Group)", "BP (Group)", "BP (Group)", "Chev…
$ MILES                 <dbl> 36.00, 539.00, 1272.00, 1206.00, 1038.00, 296.0…
$ AGE_UNKNOWN_MILES     <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,…
$ MILES_PRE_1940        <dbl> 1.0, 0.0, 152.0, 0.0, 13.0, 0.0, 0.0, 0.0, 0.0,…
$ MILES_1940            <dbl> 1, 0, 43, 8, 48, 64, 0, 0, 0, 15, 0, 729, 0, 0,…
$ MILES_1950            <dbl> 4.0, 0.0, 498.0, 486.0, 1344.0, 54.0, 243.0, 0.…
$ MILES_1960            <dbl> 7.0, 280.0, 524.0, 328.0, 222.0, 139.0, 539.0, …
$ MILES_1970            <dbl> 5.00, 631.00, 786.00, 411.00, 14.00, 104.00, 0.…
$ MILES_1980            <dbl> 2.0, 36.0, 41.0, 163.0, 19.0, 

In [22]:
test <- select(sample, COMMODITY, YEAR, OPERATOR_ID)
test <- test[duplicated(test), ]
testit::assert(nrow(test) == 0)

## 9.5 Save

In [23]:
testit::assert(nrow(subset(sample, YEAR == 2008)) > 0)
feather::write_feather(sample, paste0("../preprocessed_data/sample_consolidated_", Sys.Date(), ".feather"))
haven::write_dta(sample, paste0("../stata_data/sample_consolidated_", Sys.Date(), ".dta"))